In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    print('API key loaded')
else:
    print('API key loading issue')

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key loaded


In [7]:
system_prompt = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate, If you don't know the answer, say so.
"""

In [18]:
# checking without tool response

# def chat(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + history
#     messages.append({"role": "user", "content": message})

#     stream = openai.chat.completions.create(
#         model=MODEL,
#         messages=messages,
#         stream=True
#     )

#     response = ""
#     for chunk in stream:
#         response += chunk.choices[0].delta.content or ""
#         yield {
#             "role": "assistant",
#             "content": response,
#             "messages": history + [
#                 {"role": "user", "content": message},
#                 {"role": "assistant", "content": response}
#             ]
#         }

In [19]:
# Test without ticket price tool function

# gr.ChatInterface(
#     fn=chat,
#     chatbot=gr.Chatbot(type="messages"),
#     type="messages"
# ).launch(share=False)

# Tools

In [28]:
ticket_prices = {"london": "$799", "paris": "$899", "berlin": "$499"}

def get_ticket_price(destination_city):
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [29]:
get_ticket_price("Paris")

'$899'

In [30]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [31]:
tools = [{"type": "function", "function": price_function}]

In [46]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    # Ensure response is not None
    if response and response.choices:
        return response.choices[0].message.content
    else:
        return {"role": "assistant", "content": "I'm sorry, I couldn't process your request."}

In [47]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [49]:
# gr.ChatInterface(
#     fn=chat,
#     chatbot=gr.Chatbot(type="messages"),
#     type="messages"
# ).launch(share=False)